In [0]:
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range, input

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sortedcontainers import SortedList

from datetime import datetime
from scipy.stats import norm
from scipy.stats import multivariate_normal as mvn
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
def get_data(limit=None):
  print("Reading in and transforming data...")
  df = pd.read_csv('/content/drive/My Drive/Datasets/KNN/train.csv/train.csv')
  data = df.values
  np.random.shuffle(data)
  X = data[:, 1:] / 255.0 # data is from 0..255
  Y = data[:, 0]
  if limit is not None:
    X, Y = X[:limit], Y[:limit]
  return X, Y

In [0]:
class Bayes(object):
  def fit(self,X,Y,smoothing=10e-3):
    N,D=X.shape
    self.gaussians=dict()
    self.priors=dict()
    labels=set(Y)
    for c in labels:
      current_x=X[Y==c]
      self.gaussians[c]={
          'mean':current_x.mean(axis=0),
          'cov':np.cov(current_x.T) +np.eye(D)+ smoothing,
      }
      self.priors[c]=float(len(Y[Y==c]))/len(Y)
  def score(self,X,Y):
    P=self.predict(X)
    return np.mean(P==Y)
  def predict(self,X):
    N,D=X.shape
    K=len(self.gaussians)
    P=np.zeros((N,K))
    for c,g in iteritems(self.gaussians):
      mean,cov=g['mean'],g['cov']
      P[:,c]=mvn.logpdf(X,mean=mean,cov=cov)+ np.log(self.priors[c])
    return np.argmax(P,axis=1)

In [0]:
X,Y=get_data(10000)
Ntrain=int(len(Y)/2)
Xtrain,Ytrain=X[:Ntrain],Y[:Ntrain]
Xtest,Ytest=X[Ntrain:],Y[Ntrain:]


Reading in and transforming data...


In [0]:
model=Bayes()
t0=datetime.now()
model.fit(Xtrain,Ytrain)
print("Training time:",(datetime.now()-t0))
t0=datetime.now()
print("Train Accuracy : ",model.score(Xtrain,Ytrain) )
print("time to train accuracy",(datetime.now()-t0),"\t Train size:",len(Ytrain))
t0=datetime.now()
print("Test Accuracy : ",model.score(Xtest,Ytest) )
print("time to test accuracy",(datetime.now()-t0),"\t Test size:",len(Ytest))

Training time: 0:00:00.245040
Train Accuracy :  0.9024
time to train accuracy 0:00:04.336993 	 Train size: 5000
Test Accuracy :  0.8784
time to test accuracy 0:00:04.231323 	 Test size: 5000
